In [17]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
from tensorflow.keras import optimizers

import cv2
import math
from IPython.display import clear_output
%matplotlib inline




In [18]:
import os

print(os.listdir("../data/img_data"))
print(os.listdir("../data/video_data/test_videos"))
print(os.listdir("../"))

print(os.listdir("../ResNet-50"))
print(os.listdir("../VGG-16"))


['test', 'train']
['test1.mp4', 'test2.mp4', 'test3.mp4']
['.git', '.vscode', 'data', 'README.md', 'resnet', 'ResNet-50', 'VGG-16']
['imagenet_class_index.json', 'resnet50_weights_tf_dim_ordering_tf_kernels.h5', 'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5']
['imagenet_class_index.json', 'vgg16_weights_tf_dim_ordering_tf_kernels.h5', 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5']


In [19]:
IMG_SIZE = 224
NUM_EPOCHS = 20
NUM_CLASSES = 3
TRAIN_BATCH_SIZE = 77
TEST_BATCH_SIZE = 1 

In [20]:
NUM_CLASSES = 3

def create_model(model_size):
    my_new_model = Sequential()

    if model_size == 'L':
        resnet_weights_path = '../resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
        resnet = ResNet50(include_top=False, pooling='avg', weights='imagenet', input_shape=(224, 224, 3))
        
        my_new_model.add(resnet)
        my_new_model.layers[0].trainable = False
    else:
        vgg_weights_path = '../vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
        vgg = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3)) 
        vgg.summary()
        my_new_model.add(vgg)
        my_new_model.add(GlobalAveragePooling2D())
        my_new_model.layers[0].trainable = False
        my_new_model.layers[1].trainable = False

    my_new_model.add(Dense(NUM_CLASSES, activation='softmax'))

    opt = optimizers.Adam()
    my_new_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return my_new_model


In [21]:
def plot_history( H, NUM_EPOCHS ):
    plt.style.use("ggplot")
    fig = plt.figure()
    fig.set_size_inches(15, 5)
    
    fig.add_subplot(1, 3, 1)
    plt.plot(np.arange(0, NUM_EPOCHS), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, NUM_EPOCHS), H.history["val_loss"], label="val_loss")
    plt.title("Training Loss and Validation Loss on Dataset")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss")
    plt.legend(loc="lower left")

    
    fig.add_subplot(1, 3, 2)
    plt.plot(np.arange(0, NUM_EPOCHS), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, NUM_EPOCHS), H.history["accuracy"], label="train_acc")
    plt.title("Training Loss and Accuracy on Dataset")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    
    fig.add_subplot(1, 3, 3)
    plt.plot(np.arange(0, NUM_EPOCHS), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, NUM_EPOCHS), H.history["val_accuracy"], label="val_acc")
    plt.title("Validation Loss and Accuracy on Dataset")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")


    plt.show()
    #plt.savefig("plot.png")
    

In [1]:
NUM_CLASSES = 3

def train_model( model ):
    #ata_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
    H = model.fit_generator(
            train_generator,
            steps_per_epoch=train_generator.n/TRAIN_BATCH_SIZE,
            epochs=NUM_EPOCHS,
            validation_data=validation_generator,
            validation_steps=1 #,
            #class_weight=dict_weights
                )

    data_generator_with_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                #sear_range=0.01,
                                zoom_range=[0.9, 1.25],
                                horizontal_flip=True,
                                vertical_flip=False,
                                data_format='channels_last',
                                brightness_range=[0.5, 1.5]
                               )
                                       
    train_generator = data_generator_with_aug.flow_from_directory(
            '../data/img_data/train',
            target_size=(IMG_SIZE, IMG_SIZE),
            batch_size=TRAIN_BATCH_SIZE,
            class_mode='categorical')
    
   
    validation_generator = data_generator_with_aug.flow_from_directory(
            '../data/img_data/test',
            target_size=(IMG_SIZE, IMG_SIZE),
            batch_size=TEST_BATCH_SIZE,
            shuffle = False,
            class_mode='categorical')
    
       

    
    return model, train_generator,validation_generator

In [23]:
def get_label_dict(train_generator ):
# Get label to class_id mapping
    labels = (train_generator.class_indices)
    label_dict = dict((v,k) for k,v in labels.items())
    return  label_dict   

In [24]:
def get_labels( generator ):
    generator.reset()
    labels = []
    for i in range(len(generator)):
        labels.extend(np.array(generator[i][1]) )
    return np.argmax(labels, axis =1)

In [25]:
def get_pred_labels( test_generator):
    test_generator.reset()
    pred_vec=model.predict_generator(test_generator,
                                     steps=test_generator.n, #test_generator.batch_size
                                     verbose=1)
    return np.argmax( pred_vec, axis = 1), np.max(pred_vec, axis = 1)
    

PREDICTION


In [26]:
def draw_prediction( frame, class_string ):
    x_start = frame.shape[1] -600
    cv2.putText(frame, class_string, (x_start, 75), cv2.FONT_HERSHEY_SIMPLEX, 2.5, (255, 0, 0), 2, cv2.LINE_AA)
    return frame

In [27]:
def prepare_image_for_prediction( img):
   
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    # The below function inserts an additional dimension at the axis position provided
    img = np.expand_dims(img, axis=0)
    # perform pre-processing that was done when resnet model was trained.
    return preprocess_input(img)

In [28]:
model = create_model('L')

In [29]:
trained_model_l, train_generator,validation_generator = train_model(model)
label_dict_l = get_label_dict(train_generator )

Found 693 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


C:\Users\amrit\AppData\Local\Temp\ipykernel_18012\1846641491.py:32: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  H = model.fit_generator(


Epoch 1/20
9/9 [==============================] - 40s 4s/step - loss: 0.8340 - accuracy: 0.6623 - val_loss: 2.2055 - val_accuracy: 0.0000e+00
Epoch 2/20
9/9 [==============================] - 35s 4s/step - loss: 0.4599 - accuracy: 0.8384 - val_loss: 0.2700 - val_accuracy: 1.0000
Epoch 3/20
9/9 [==============================] - 36s 4s/step - loss: 0.3274 - accuracy: 0.8874 - val_loss: 2.6188 - val_accuracy: 0.0000e+00
Epoch 4/20
9/9 [==============================] - 35s 4s/step - loss: 0.2711 - accuracy: 0.9033 - val_loss: 0.5240 - val_accuracy: 1.0000
Epoch 5/20
9/9 [==============================] - 36s 4s/step - loss: 0.2237 - accuracy: 0.9192 - val_loss: 1.3273 - val_accuracy: 0.0000e+00
Epoch 6/20
9/9 [==============================] - 35s 4s/step - loss: 0.2119 - accuracy: 0.9163 - val_loss: 0.1552 - val_accuracy: 1.0000
Epoch 7/20
9/9 [==============================] - 35s 4s/step - loss: 0.1819 - accuracy: 0.9307 - val_loss: 0.6156 - val_accuracy: 1.0000
Epoch 8/20
7/9 [======

: 

In [ ]:
def get_display_string(pred_class, label_dict):
    txt = ""
    for c, confidence in pred_class:
        txt += label_dict[c]
        if c :
            txt += '['+ str(confidence) +']'
    return txt

In [ ]:

import json


generator_config = {
    'directory': '../data/img_data/train',
    'target_size': (IMG_SIZE, IMG_SIZE),
    'batch_size': TRAIN_BATCH_SIZE,
    'class_mode': 'categorical'
}

with open('train_generator_config.json', 'w') as config_file:
    json.dump(generator_config, config_file)
    

In [ ]:
train_generator

trained_model_l.save('trained_model_l.h5')



In [ ]:
video_path = '../data/video_data/test_videos/test1.mp4'
predict ( trained_model_l, video_path, 'test1_9.avi',  label_dict_l) 

1/1 [==============================] - 0s 98ms/step


In [ ]:

def predict(  model, video_path, filename, label_dict ):
    
    vs = cv2.VideoCapture(video_path)
    fps = math.floor(vs.get(cv2.CAP_PROP_FPS))
    ret_val = True
    writer = 0
    
    while True:
        ret_val, frame = vs.read()
        if not ret_val:
            break
       
        resized_frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        frame_for_pred = prepare_image_for_prediction( resized_frame )
        pred_vec = model.predict(frame_for_pred)
        #print(pred_vec)
        pred_class =[]
        confidence = np.round(pred_vec.max(),2) 
        
        if confidence > 0.4:
            pc = pred_vec.argmax()
            pred_class.append( (pc, confidence) )
        else:
            pred_class.append( (0, 0) )
        if pred_class:
            txt = get_display_string(pred_class, label_dict)       
            frame = draw_prediction( frame, txt )

        if not writer:
            fourcc = cv2.VideoWriter_fourcc(*"XVID")
            writer = cv2.VideoWriter(filename, fourcc, fps,(frame.shape[1], frame.shape[0]), True)
            
        # write the out
        writer.write(frame)
        
    vs.release()
    writer.release()
      
   

In [ ]:
video_path = '../data/video_data/test_videos/test1.mp4'
predict ( trained_model_l, video_path, 'test1_9.avi',  label_dict_l) 

1/1 [==============================] - 0s 98ms/step


In [ ]:
predict(trained_model_l,video_path)

In [ ]:
video_path = '../data/video_data/test_videos/test3.mp4'
predict ( trained_model_l, video_path, 'test3_9.avi',  label_dict_l) 

1/1 [==============================] - 0s 221ms/step
